# Imports

In [114]:
# #############################################################################
# ####### UNCOMMENT AND RUN THIS SECTION FIRST TIME THE NOTEBOOK IS RUN #######
# #############################################################################

# # imoprt nltk and make sure models are up to date
# import ssl
# import nltk
# ssl._create_default_https_context = ssl._create_unverified_context
# nltk.download()

In [115]:
# user libraries
from user_library.visualize.visualize import plot_panel

# other dependencies
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from functools import reduce
from linearmodels.panel import PanelOLS
import os, warnings, itertools
from wordcloud import WordCloud
import plotly.express as px
import plotly.io as pio
from collections import Counter

# Query all data

### This section downloads/queries and prepares all relevant data. All data is downloaded to default directories. The process takes approx. 30 min.

In [116]:
# set company tickers
companies = ['cpb', 'gm', 'dell', 'pg', 'pep']
sec_filings = ['10-K', '10-Q']

In [117]:
# #############################################################################
# ####### UNCOMMENT AND RUN THIS SECTION FIRST TIME THE NOTEBOOK IS RUN #######
# #############################################################################

# # import neccessary user modules
# from user_library.news_scrape.cnn_scrape import scrapeCNN
# from user_library.sec_query.download import download, extract
# from user_library.sec_query.transform import transform
# from user_library.sec_query.query import query
# from user_library.stock_price_query.get_prices import get_prices

# # scrape cnn for articles
# scrapeCNN('2020-01-01', '2020-12-31', 200)

# # download and extract sec data
# download('2019q1', '2021q4')
# extract()

# # tranform and query sec data
# transform()
# query('2019q1', '2021q4', sec_filings, companies)

# # query stock prices
# get_prices(companies, '2019-01-01', '2021-12-31')

# Transorm all data

### Run major transformations on data and store in new folder that can be pushed to github, thus avoiding the upload of large files and make sure the notebook runs with out retrieval of raw data first.

In [118]:
# #############################################################################
# ####### UNCOMMENT AND RUN THIS SECTION FIRST TIME THE NOTEBOOK IS RUN #######
# #############################################################################

# # import neccessary user modules
# from user_library.stmt_preprocessing.get import GetFinancialStatements, stmt_performance
# from user_library.news_scrape.clean import clean_text

# # import sec data
# path_to_sec_data = r'sec_data/queries/merged.parquet'
# sec_data = GetFinancialStatements(path_to_sec_data)

# # run transformation to get relevant perf. indicators
# sec_perf_metrics = stmt_performance(companies, sec_data)
# sec_perf_metrics.columns = sec_perf_metrics.columns.set_levels(
#     sec_perf_metrics.columns.levels[1].str.upper(), 
#     level=1
# )

# # save df
# if not os.path.isdir('data'):
#     os.mkdir('data')
# sec_perf_metrics.to_parquet('data/sec_performance_metrics.parquet')

# # import cnn data
# path_to_cnn_data = r'cnn_data/raw.parquet'
# cnn_data = pd.read_parquet(path_to_cnn_data).dropna().reset_index(drop=True).\
#     rename(columns={'Body':'Corpus'})

# # clean data
# cnn_data['Headline (clean)'] = cnn_data['Headline'].apply(lambda x: clean_text(x))
# cnn_data['Corpus (clean)'] = cnn_data['Corpus'].apply(lambda x: clean_text(x))

# # transform the datetime column to year and quarter 
# cnn_data['Year'] = cnn_data['Date'].dt.year
# cnn_data['Quarter'] = cnn_data['Date'].dt.quarter
# cnn_data = cnn_data.drop(columns='Date')

# # save df
# cnn_data.to_parquet('data/cnn_articles.parquet')

# # import price data
# path_to_price_data = r'price_data/historical.parquet'
# price_data = pd.read_parquet(path_to_price_data)
# pct_change_data = price_data.set_index('Date').pct_change().dropna()

# # save df
# pct_change_data.to_parquet('data/pct_change.parquet')

# Run sentiment analysis on cnn articles

In [119]:
# import data
cnn_articles = pd.read_parquet('data/cnn_articles.parquet')

# run analysis
sia = SentimentIntensityAnalyzer()
cnn_sentiment = cnn_articles.drop(columns=['Headline', 'Corpus'])
cnn_sentiment['Score (headline)'] = cnn_sentiment['Headline (clean)']\
    .apply(lambda x: sia.polarity_scores(x))
cnn_sentiment['Score (corpus)'] = cnn_sentiment['Corpus (clean)']\
    .apply(lambda x: sia.polarity_scores(x))
    
# unpack score dict
cnn_sentiment = pd.concat(
    [
        cnn_sentiment.drop(columns=['Score (headline)', 'Score (corpus)']),
        pd.DataFrame.from_records(cnn_sentiment['Score (headline)'].to_list()).add_suffix(' (headline)'),
        pd.DataFrame.from_records(cnn_sentiment['Score (corpus)'].to_list()).add_suffix(' (corpus)'),
    ],
    axis=1
)

In [120]:
# produce agg stats and create latex table
cnn_sentiment_stats = cnn_sentiment.drop(columns=['Headline (clean)', 'Corpus (clean)']).\
    groupby(['Year', 'Quarter']).\
        agg(['mean', 'min', 'max', 'std', 'count']).\
            reset_index().\
                melt(id_vars=['Year', 'Quarter'], var_name=['Score', 'Stat'])
                
# create new indexes
unicode_cnn = dict.fromkeys(map(ord, ['(', ')']), '')
cnn_sentiment_stats['Part'] = cnn_sentiment_stats.Score.apply(
    lambda x: x.split(' ')[1].translate(unicode_cnn)
)
cnn_sentiment_stats['Score'] = cnn_sentiment_stats.Score.apply(
    lambda x: x.split(' ')[0]
)
cnn_sentiment_stats = round(cnn_sentiment_stats, 3).pivot_table(
    index=['Year', 'Quarter'], 
    columns=['Part', 'Score', 'Stat'], 
    values='value'
).transpose()

In [121]:
# create latex table
if not os.path.isdir('LaTex'):
    os.mkdir('LaTex')

cnn_sentiment_stats_tex = cnn_sentiment_stats.to_latex(
    float_format='%.3f',
    column_format=r'|r|r|rrrr|',
    position='h',
    header=True,
    label='sentStatsExp',
    caption='Summary statistics of the sentiment analysis on CNN articles',
    bold_rows=True,
    index=True,
    longtable=True
)

with open('Latex/sentiment_stats_exp.txt', 'w') as file:
    file.write(cnn_sentiment_stats_tex)

In [122]:
# crate table with only compound stats for reg
cnn_sentiment_stats_compund = cnn_sentiment_stats[
    (cnn_sentiment_stats.index == ('corpus', 'compound',  'mean')) | \
        (cnn_sentiment_stats.index == ('headline', 'compound',  'mean'))
]
cnn_sentiment_stats_compund = cnn_sentiment_stats_compund.reset_index().\
    melt(id_vars=['Part', 'Score', 'Stat'])

# Run regressions

In [123]:
# load data
pct_change = pd.read_parquet('data/pct_change.parquet')
sec_performance_metrics = pd.read_parquet('data/sec_performance_metrics.parquet')

In [124]:
# prepare quarterly returns
pct_change_q = pct_change.resample('Q').sum().reset_index()
pct_change_q['Year'] = pct_change_q.Date.dt.year
pct_change_q['Quarter'] = pct_change_q.Date.dt.quarter

In [135]:
# prepare sec panel data
panel = sec_performance_metrics
panel = reduce(
    lambda left, right: pd.merge(
        left, 
        right,
        on=['Year', 'Quarter', 'Ticker'],
        how='left'
    ),
    [
        panel[y].reset_index().\
            melt(id_vars=['Year', 'Quarter'], var_name='Ticker', value_name=y) \
                for y in sec_performance_metrics.columns.get_level_values(level=0).unique()
    ]
)
panel = panel.merge(
    pct_change_q.drop(columns='Date').\
        melt(id_vars=['Year', 'Quarter'], var_name='Ticker', value_name='Return'),
        how='left',
        on=['Year', 'Quarter', 'Ticker']
)
panel = panel.merge(
    cnn_sentiment_stats_compund.pivot(
        index=['Year', 'Quarter'], 
        columns='Part', 
        values='value'
    ).reset_index(),
    how='left',
    on=['Year', 'Quarter']
)

In [136]:
# prettify the table
panel_trans = {
    'Contrib. Mgn.':'Contrib. Mgn., Ratio',
    'D/E, Ratio':'D/E',
    'corpus':'Corpus',
    'headline':'Headline'
}
panel.columns = [
    x.replace(', Ratio', '').\
        replace('corpus', 'Corpus').\
            replace('headline', 'Headline') for x in panel.columns
]
panel = panel.dropna()
panel['OFC'] = panel.OFC / 10e+8
panel

,Year,Quarter,Ticker,Contrib. Mgn.,D/E,OFC,Return,Corpus,Headline
3,2020,2,CPB,0.142792,0.797059,1.125,0.093190,0.164,-0.029
4,2020,3,CPB,0.127373,0.792354,1.396,-0.010505,0.215,0.004
5,2020,4,CPB,0.197009,0.776677,0.180,0.010966,0.312,0.026
11,2020,2,DELL,0.032059,0.969385,-0.796,0.362256,0.164,-0.029
12,2020,3,DELL,0.041183,0.957386,2.536,0.224843,0.215,0.004
13,2020,4,DELL,0.043561,0.945640,5.530,0.086928,0.312,0.026
19,2020,2,GM,-0.011255,0.816899,-1.254,0.253580,0.164,-0.029
20,2020,3,GM,0.045512,0.799692,9.977,0.180872,0.215,0.004
21,2020,4,GM,0.054162,0.788783,16.670,0.359608,0.312,0.026
27,2020,2,PEP,0.142258,0.859402,1.462,0.114377,0.164,-0.029


In [137]:
# run regression
warnings.simplefilter(action='ignore', category=UserWarning)
panel_reg = panel
unicode = dict.fromkeys(map(ord, [' ', '/']), '_')
columns = panel_reg.columns
new_columns = dict(zip(columns, [x.translate(unicode) for x in columns]))
panel_reg = panel_reg.rename(columns=new_columns)
panel_reg['Date'] = pd.to_datetime(
    panel_reg['Year'].astype(str) + 'Q' + panel_reg['Quarter'].astype(str)
)
panel_reg.drop(['Year', 'Quarter'], axis=1, inplace=True)
panel_reg = panel_reg.set_index(['Ticker', 'Date'])
Y = panel_reg.columns[:-2]
X = panel_reg.columns[-2:]
reg_res = []
for y in Y:
    for x in X:
        model = PanelOLS(
            dependent=panel_reg[y],
            exog=panel_reg[x],
            entity_effects=False
        )
        res = model.fit()
        reg_res.append(
            pd.DataFrame(
                {
                    'Y':y,
                    'X': x,
                    'Coefficient': res.params[x],
                    'Std. error': res.std_errors,
                    'p-value': res.pvalues
                }
            )
        )
reg_df = pd.concat(reg_res).reset_index(drop=True)
reg_df['Y'] = reg_df.Y.apply(lambda x: x.replace('_', ' ').\
    replace('D E', 'D/E').\
        replace('OFC', 'OFC (10e+9)'))
reg_df = reg_df.set_index(['Y', 'X'])
reg_df

Coefficient  Std. error       p-value
Y             X                                              
Contrib. Mgn. Corpus       0.522816    0.097786  1.030978e-04
              Headline     0.750029    1.787206  6.810987e-01
D/E           Corpus       3.238264    0.272439  1.056917e-08
              Headline     0.182545    9.567671  9.850471e-01
OFC (10e+9)   Corpus      25.498017    6.117969  9.482552e-04
              Headline    91.439008   93.427685  3.443317e-01
Return        Corpus       0.531239    0.142288  2.224579e-03
              Headline    -1.320473    2.090039  5.377026e-01

In [138]:
# reg res to latex
reg_df_tex = reg_df.to_latex(
    float_format='%.3f',
    column_format=r'|rr|rrr|',
    position='h',
    header=True,
    label='regRes',
    caption='Regression results',
    bold_rows=True,
    index=True
)
with open('LaTex/regression_results', 'w') as file:
    file.write(reg_df_tex)

# Visualize

In [129]:
# create 3d plots
panel_plot = panel
panel_plot['Timestamp'] = pd.to_datetime(
    panel_plot['Year'].astype(str) + 'Q' + panel_plot['Quarter'].astype(str)
)
panel_plot.drop(['Year', 'Quarter'], axis=1, inplace=True)
X_plot = panel_plot.columns[-3:-1]
Y_plot = panel_plot.columns[1:-3]
for y in Y_plot:
    for x in X_plot:
        plot_panel(panel_plot, x, y, 'Ticker', save=True)

In [130]:
# create word cloud funcctions
def analyze_sentiment(text):
    
    tokens = word_tokenize(text)
    word_counts = Counter(tokens)
    positive_words = []
    negative_words = []

    for token, count in word_counts.items():
        
        scores = sia.polarity_scores(token)
        
        if scores['pos'] > scores['neg']:
            positive_words.extend([token] * count)
        elif scores['neg'] > scores['pos']:
            negative_words.extend([token] * count)
    
    return positive_words, negative_words

def sort_dict_by_value(d):
    return {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}

In [131]:
# run on body
pd.options.mode.chained_assignment = None
word_cloud_df = cnn_sentiment[['Corpus (clean)', 'Year', 'Quarter']]
word_cloud_df['Positive'], word_cloud_df['Negative'] = \
    zip(*word_cloud_df['Corpus (clean)'].apply(analyze_sentiment))

In [132]:
# count
word_cloud_grouped = word_cloud_df.groupby(['Year', 'Quarter'])\
    [['Positive', 'Negative']].\
        apply(lambda x: pd.Series(
            {
                'Count (pos)': sort_dict_by_value(
                    Counter(word for words in x['Positive'] for word in words)
                ),
                'Count (neg)': sort_dict_by_value(
                    Counter(word for words in x['Negative'] for word in words)
                )
            }
        )
).reset_index()

In [133]:
# make plot
for score in ['pos', 'neg']:
    for q in [0, 1, 2]:
        cloud = WordCloud(
            width=700, 
            height=300, 
            background_color='white', 
            colormap='twilight', 
            font_path='user_library/visualize/cmunrm.ttf'
        ).\
            generate_from_frequencies(
                dict(
                    itertools.islice(word_cloud_grouped[f'Count ({score})'][q].items(), 50)
                )
            )
        fig = px.imshow(cloud)
        fig.update_layout(
            title_text=f'Q{q+2}', 
            title_x=0.5, 
            title_font=dict(family="Serif", size=20),
            margin=dict(l=0, r=0, b=10, t=90),
            xaxis=dict(showticklabels=False),
            yaxis=dict(showticklabels=False),
            autosize=False 
        )
        fig.update_xaxes(showticklabels=False)
        fig.update_yaxes(showticklabels=False)
        title = f'{score}_Q{q+2}.png'
        if not os.path.isdir('images/wordcloud'):
            os.mkdir('images/wordcloud')
        pio.write_image(fig, f'images/wordcloud/{title}')